<a href="https://colab.research.google.com/github/AmyArora222/Plant-Disease-Detection/blob/main/PlantDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Upload Zip File


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload apple.zip


Extract Contents of Zip File


In [ ]:
import zipfile

for fname in uploaded.keys():
    with zipfile.ZipFile(fname, 'r') as zip_ref:
        zip_ref.extractall(".")


Function to Train a Disease Classification Model for a Crop

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import load_model
import json

img_size = (128, 128)
batch_size = 32

def train_crop_model(crop_folder):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_gen = datagen.flow_from_directory(
        crop_folder,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_gen = datagen.flow_from_directory(
        crop_folder,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(train_gen.num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    print(f"\nTraining model for: {crop_folder}")
    model.fit(train_gen, epochs=10, validation_data=val_gen)

    # Save model
    model.save(f"{crop_folder}_model.h5")

    # Save class mapping
    class_mapping = train_gen.class_indices
    with open(f"{crop_folder}_classes.json", "w") as f:
        json.dump(class_mapping, f)


Train Disease Classification Models for Different Crops

In [ ]:
train_crop_model("apple")
train_crop_model("potato")
train_crop_model("corn")
train_crop_model("grape")
train_crop_model("peach")
train_crop_model("cherry")
train_crop_model("bellpepper")
train_crop_model("tomato")
train_crop_model("strawberry")

Function to Load the Trained Model and make a Prediction on a Test Image

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_disease(crop_name, img_path):
    model = load_model(f"{crop_name}_model.h5")
    with open(f"{crop_name}_classes.json", "r") as f:
        class_indices = json.load(f)

    index_to_class = {v: k for k, v in class_indices.items()}

    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    pred_class = np.argmax(pred)
    predicted_label = index_to_class[pred_class]

    print(f"Predicted: {predicted_label}")


Upload a Test Image for Prediction

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload test image


Predict the Disease

In [ ]:
predict_disease("tomato", "/content/tomato.JPG")  # replace with actual filename